In [ ]:
from langgraph.graph import END, START, StateGraph
from typing_extensions import Optional, TypedDict

from src.llm.llm import llm


# 定义图的状态（保存中间结果）
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: Optional[str]  # 标记为可选，因为可能不会生成
    final_joke: Optional[str]  # 标记为可选


# 第一步：生成初始笑话
def generate_joke(state: State):
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}


# 质量检查
def check_punchline(state: State):
    if "?" in state["joke"] or "!" in state["joke"]:
        return "pass"  # 小写更符合惯例
    return "fail"


# 第二步：改进笑话
def improve_joke(state: State):
    msg = llm.invoke(f"Make this joke funnier: {state['joke']}")
    return {"improved_joke": msg.content}


# 第三步：最后打磨
def polish_joke(state: State):
    msg = llm.invoke(f"Add a twist: {state['improved_joke']}")
    return {"final_joke": msg.content}


# 如果检查失败，直接将原笑话作为最终结果
def use_original(state: State):
    return {"improved_joke": state["joke"], "final_joke": state["joke"]}


# 构建工作流
workflow = StateGraph(State)

# 添加节点
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)
workflow.add_node("use_original", use_original)  # 备用节点

# 连接节点
workflow.add_edge(START, "generate_joke")
workflow.add_conditional_edges(
    "generate_joke",
    check_punchline,
    {"pass": "improve_joke", "fail": "use_original"},  # Fail 时使用原笑话
)
workflow.add_edge("improve_joke", "polish_joke")
workflow.add_edge("use_original", END)
workflow.add_edge("polish_joke", END)

# 编译工作流
chain = workflow.compile()

# 执行
result = chain.invoke({"topic": "猫"})
print("最终笑话:", result["final_joke"])
print("完整结果:", result)

最终笑话: 好的，这里有一个关于猫的中文短笑话：

**猫主子偷吃了鱼干，被抓包后一脸淡定，对着铲屎官翻了个白眼：**  
“喵嗷！你瞅啥？本喵只是帮你们鉴定一下鱼干过期没！”
完整结果: {'topic': '猫', 'joke': '好的，这里有一个关于猫的中文短笑话：\n\n**猫主子偷吃了鱼干，被抓包后一脸淡定，对着铲屎官翻了个白眼：**  \n“喵嗷！你瞅啥？本喵只是帮你们鉴定一下鱼干过期没！”', 'improved_joke': '好的，这里有一个关于猫的中文短笑话：\n\n**猫主子偷吃了鱼干，被抓包后一脸淡定，对着铲屎官翻了个白眼：**  \n“喵嗷！你瞅啥？本喵只是帮你们鉴定一下鱼干过期没！”', 'final_joke': '好的，这里有一个关于猫的中文短笑话：\n\n**猫主子偷吃了鱼干，被抓包后一脸淡定，对着铲屎官翻了个白眼：**  \n“喵嗷！你瞅啥？本喵只是帮你们鉴定一下鱼干过期没！”'}
